In [1]:
import pandas as pd
from utils import *

config = load_config()

In [2]:
PROJECT_PATH = config.project_path
OUTPUT_PATH = PROJECT_PATH.joinpath("outputs")

def load_output(model, topn, inference) :
    with open(OUTPUT_PATH.joinpath(f"{model}_{topn}_{inference}_output.txt"), 'r') as f :
        output = f.read()
    return output

In [3]:
text = load_output("biomistral7b_avigon", "top3", "fewshot")
print(text)


diabetes.report178889.txt ================================= 


precison : 0.0
recall : 0.0
f1_score : 0
mrr : 0

gold : 
 1. tinnitus
1. whipple procedure
2. insulin-dependent diabetic
2. simvastatin
2. reinopathy
2. microalbumin
3. colon adenomas
3. colonoscopy

 
pred : 
 1. pancreatic neoplasm

 
pred_text : [('1', 'pancreatic neoplasm')]
diabetes.report391725.txt ================================= 


precison : 1.0
recall : 0.714
f1_score : 0.833
mrr : 0

gold : 
 1. expressive aphasia
2. tia
2. hypertensive emergency
2. paroxysmal atrial fibrillation
3. secondary prophylaxis

 
pred : 
 1. tia 2. hypertensive emergency 3. hypertension 4. cerebral small vessel disease 5. left atrial enlargement 6. paroxysmal atrial fibrillation 7. event monitor 8. cardiac study 9. aspirin 81 mg 10. optimization of other vascular risk factors 11. blood pressure 130/80 12. 911 13. 6 months 14. left atrial enlargement 15. paroxysmal atrial fibrillation 16. stroke symptoms 17. 30-day event monitor 18. 

In [28]:
import re
text = "1. tia 2. hypertensive emergency 2.2 no 3. hypertension 4. cerebral small vessel disease 5. left atrial enlargement 6. paroxysmal atrial fibrillation 7. event monitor 8. cardiac study 9. aspirin 81 mg 10. optimization of other vascular risk factors 11. blood pressure 130/80 12. 911 13. 6 months 14. left atrial enlargement 15. paroxysmal atrial fibrillation 16. stroke symptoms 17. 30-day event monitor 18. counseled 19. stroke workup 20. followup 21. phone 22. referral 23. questions 24. contact 25. no 26. questions 2"

# p = re.compile("(\d+)\. (\w+)")
p = re.compile('(\d+)\.\d?\.?\s(.+)')
# p = re.split('(\d+)\.?', text)
p

re.compile(r'(\d+)\.\d?\.?\s(.+)', re.UNICODE)

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import pipeline

model_path = config.model_path("mistral7b")
model = pipeline("text-generation", model_path, max_length=1000, device_map="auto")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
prompt = """
# Instruction 
Here is the output of a model. 
I want you to reformat this output as follows :

1. contents 
2. contents 
3. contents
...

# Output
{output}

# Response
"""

In [21]:
text = "1. appendiceal adenocarcinoma with peritoneal carcinomatosis. 1.1. ct scan of the abdomen and pelvis with contrast. 1.2. pet scan of the abdomen and pelvis. 2. diabetes. 2.1. hemoglobin a1c. 3. fibromyalgia. 3.1. rheumatology consult. 4. hypertension. 4.1. 24-h urine sodium. 4.2. echocardiogram. 5. osteoarthritis. 5.1. joint aspiration and injection. 6. obstructive sleep apnea. 6.1. cpap. 7. peripheral neuropathy. 7.1. neurology consult. 8. restless leg"


In [22]:
out = model(prompt.format(output=text), max_new_tokens=400, return_full_text=False)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [24]:
print(out[0]['generated_text'])

1. Appendiceal adenocarcinoma with peritoneal carcinomatosis
   1.1. CT scan of the abdomen and pelvis with contrast
   1.2. PET scan of the abdomen and pelvis
2. Diabetes
   2.1. Hemoglobin A1c
3. Fibromyalgia
   3.1. Rheumatology consult
4. Hypertension
   4.1. 24-hour urine sodium
   4.2. Echocardiogram
5. Osteoarthritis
   5.1. Joint aspiration and injection
6. Obstructive sleep apnea
   6.1. CPAP
7. Peripheral neuropathy
   7.1. Neurology consult
8. Restless leg syndrome


In [29]:
p.findall(out[0]['generated_text'])

[('1', 'Appendiceal adenocarcinoma with peritoneal carcinomatosis'),
 ('1', 'CT scan of the abdomen and pelvis with contrast'),
 ('1', 'PET scan of the abdomen and pelvis'),
 ('2', 'Diabetes'),
 ('2', 'Hemoglobin A1c'),
 ('3', 'Fibromyalgia'),
 ('3', 'Rheumatology consult'),
 ('4', 'Hypertension'),
 ('4', '24-hour urine sodium'),
 ('4', 'Echocardiogram'),
 ('5', 'Osteoarthritis'),
 ('5', 'Joint aspiration and injection'),
 ('6', 'Obstructive sleep apnea'),
 ('6', 'CPAP'),
 ('7', 'Peripheral neuropathy'),
 ('7', 'Neurology consult'),
 ('8', 'Restless leg syndrome')]